# Semantic similarity from binary vector encodings

The standard definitions for semantic similarity are typically based on some notion of sets, and operations on them (specifically, length, intersection, union). For example, the Jaccard similarity between two ontology concepts (terms) $C_1$ and $C_2$ is defined as the following:
$$SimJ(C_1,C_2)=\frac{|S(C_1)\cap S(C_2)|}{|S(C_1)\cup S(C_2)|}$$
where $S(C)$ is the set of subsumers of $C$ (concepts considered subsuming concept $C$, directly or transitively): $\forall D\in S(C): C\sqsubseteq D$. (Hence, $C$ is always included in $S(C)$.

## Motivation
Set operations typically take loops to evaluate, and thus cannot easily take advantage of vector and/or matrix alegbra (and the accelerations available to those).

However, by using a binary vector encoding we can use fast vector algebra for most (all?) semantic similarity metric calculatiobs. A binary vector encoding **S** for a concept _C_ is the following:
$$\mathbf{S}_C: s_i=\begin{cases}1\;\mathrm{if}\ C_i\in S(C)\\ 0\; \mathrm{otherwise}\end{cases}$$
where $C_i,\dots,C_n$ are ontology terms. $C_i,\dots,C_n$ could be all concepts in the ontology; but include at least all subsumers of $C$ and all other concepts that are to be compared.

Here we will see how we can obtain such an encoding from a triple store, starting with a SPARQL query.

## Obtain adjancy table

In [ ]:
import pandas as pd
import numpy as np
from SPARQLWrapper import get_sparql_dataframe

SPARQL_ENDP = "https://ubergraph.apps.renci.org/sparql"

In [ ]:
adjTable = get_sparql_dataframe(endpoint=SPARQL_ENDP, query="""
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX renci: <http://reasoner.renci.org/vocab/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT DISTINCT ?fin ?finLabel ?super ?ic
FROM <http://reasoner.renci.org/ontology>
FROM <http://reasoner.renci.org/nonredundant>
WHERE {
  VALUES ?fin {
    obo:UBERON_0000151 obo:UBERON_0000152 obo:UBERON_4000163 obo:UBERON_4000164
    obo:UBERON_0003097 obo:UBERON_2000251 obo:UBERON_0002102 obo:UBERON_0002103
  }
  ?fin rdfs:label ?finLabel ;
       rdfs:subClassOf* ?super .
  ?super rdf:type owl:Class .
  ?super renci:normalizedSubClassInformationContent ?ic .
}
""")

In [ ]:
adjTable

## Cross-tabulate to obtain binary encoded vectors

In [ ]:
subsumM = pd.crosstab(adjTable["finLabel"], adjTable["super"])

In [ ]:
subsumM.T

In [ ]:
subsumM.loc["pectoral fin",]

## Using vector algebra on binary encoded vectors

We can now use standard vector algebra operations to obtain the number of common subsumers (i.e. ancestors) for two concepts as the dot product:

In [ ]:
np.sum(subsumM.loc["pectoral fin",] * subsumM.loc["adipose fin",])

Remember the equation for deriving the Jaccard similarity from binary vectors:
$$SimJ(C_1,C_2)=\frac{\mathbf{S}_{C_1}\cdot\mathbf{S}_{C_2}}{||\mathbf{S}_{C_1}||^2+||\mathbf{S}_{C_2}||^2-\mathbf{S}_{C_1}\cdot\mathbf{S}_{C_2}}$$

The dot product of each pair of vectors is the numerator (i.e., number of ancestors in common). We can compute these using matrix dot product:

In [ ]:
simM = np.dot(subsumM, subsumM.T)

The diagonal of the resulting matrix are the numbers of subsumers (i.e., ancestors) for each vector-encoded term:

In [ ]:
nsubsum = np.diag(simM)

In [ ]:
simM

In [ ]:
nsubsum

For the denominator we can start with the negative of the numerator, then add the diagonal as row vector and as column vector:

In [ ]:
denom = -simM

In [ ]:
denom = ((denom + nsubsum).T + nsubsum).T

In [ ]:
simJC = simM / denom

In [ ]:
pd.DataFrame(simJC, index=subsumM.index, columns=subsumM.index)

## Using semantic similarity for clustering

Remember that $1-SimJ$ is a proper distance metric (satisfies the triangle inequality).

In [ ]:
import scipy.cluster.hierarchy as hclust
import scipy.spatial.distance as dist
import matplotlib.pyplot as plt

In [ ]:
# Convert the distance matrix to a condensed form (required for hierarchical clustering)
distM = dist.squareform(1 - simJC)

# Perform hierarchical/agglomerative clustering
Z = hclust.linkage(distM, method='average')  # You can also use 'single', 'average', etc.

# Create a dendrogram
plt.figure(figsize=(10, 6))
hclust.dendrogram(Z, labels=subsumM.index)  # Add labels if desired
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Anatomical Entity')
plt.ylabel('Distance')
plt.show()

## Using SciPy distance metrics

We can also use the binary-encoded term vectors to obtain pairwise distances from the SciPy library. This includes Jaccard.

In [ ]:
distM = dist.pdist(subsumM, metric="jaccard")

In [ ]:
dist.squareform(distM) - (1-simJC)

In [ ]:
# Perform hierarchical/agglomerative clustering
Z = hclust.linkage(distM, method='average')  # You can also use 'single', 'average', etc.

# Create a dendrogram
plt.figure(figsize=(10, 6))
hclust.dendrogram(Z, labels=subsumM.index)  # Add labels if desired
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Anatomical Entity')
plt.ylabel('Distance')
plt.show()